In [2]:
import shapefile as shp
import json

In [3]:
s = shp.Reader("shapefiles/Builtup_Kalonda.shp")

In [4]:
fields = s.fields[1:]
f = fields[0]
fields, [f[0] for f in fields]

([['OBJECTID', 'N', 10, 0],
  ['GlobalID', 'C', 38, 0],
  ['Uniq_Id', 'C', 50, 0],
  ['State_Code', 'C', 2, 0],
  ['District_C', 'C', 3, 0],
  ['Tehsil_Cod', 'C', 5, 0],
  ['Block_Code', 'C', 5, 0],
  ['Village_Co', 'C', 6, 0],
  ['LGD_Code', 'C', 6, 0],
  ['Built_Up_A', 'N', 5, 0],
  ['Village_Na', 'C', 50, 0],
  ['Area_Desc', 'C', 50, 0],
  ['Name', 'C', 50, 0],
  ['Owner_Name', 'C', 50, 0],
  ['Property_I', 'C', 50, 0],
  ['Property_C', 'C', 50, 0],
  ['Area_Sqm', 'F', 19, 11],
  ['Roof_type', 'N', 5, 0],
  ['No_Floors', 'N', 5, 0],
  ['Remarks', 'C', 50, 0],
  ['Adl_Info', 'C', 50, 0],
  ['SHAPE_Leng', 'F', 19, 11],
  ['SHAPE_Area', 'F', 19, 11],
  ['GP_Code', 'C', 6, 0],
  ['GP_Name', 'C', 50, 0]],
 ['OBJECTID',
  'GlobalID',
  'Uniq_Id',
  'State_Code',
  'District_C',
  'Tehsil_Cod',
  'Block_Code',
  'Village_Co',
  'LGD_Code',
  'Built_Up_A',
  'Village_Na',
  'Area_Desc',
  'Name',
  'Owner_Name',
  'Property_I',
  'Property_C',
  'Area_Sqm',
  'Roof_type',
  'No_Floors',
  '

In [5]:
shapes = s.shapes()
sh = shapes[0]

In [6]:
sh.bbox, sh.parts, sh.points, sh.shapeType

([8646538.384300001, 3317213.4416000023, 8646543.103, 3317220.3533000024],
 [0],
 [(8646538.384300001, 3317219.283399999),
  (8646539.8134, 3317220.3532999977),
  (8646543.103, 3317214.365699999),
  (8646541.131499998, 3317213.4416000023),
  (8646539.5948, 3317216.9070999995),
  (8646538.384300001, 3317219.283399999)],
 5)

In [7]:
records = s.records()
r=records[0]
r.GP_Code

'63317'

In [23]:
data = []
for i in range(len(records)):
    one = {}
    points = []
    one["points"] = [{"x": p[0], "y": p[1]} for p in shapes[i].points]
    one["bbox"] = {
        "x1": shapes[i].bbox[0],
        "y1": shapes[i].bbox[1],
        "x2": shapes[i].bbox[2],
        "y2": shapes[i].bbox[3],
    }
    for j in range(len(fields)):
        one[fields[j][0]] = records[i][j]
    data.append(one)
# print(data[0])
json.dump({"structures": data}, open("village_data.json", "w"))
# with open('village_data.json', 'w') as outfile:
#     outfile.write(str(data).replace("'", '"'))

In [18]:
d = json.load(open("village_data.json"))
d

[{'points': [{'X': 8646538.384300001, 'Y': 3317219.283399999},
   {'X': 8646539.8134, 'Y': 3317220.3532999977},
   {'X': 8646543.103, 'Y': 3317214.365699999},
   {'X': 8646541.131499998, 'Y': 3317213.4416000023},
   {'X': 8646539.5948, 'Y': 3317216.9070999995},
   {'X': 8646538.384300001, 'Y': 3317219.283399999}],
  'OBJECTID': 52390,
  'GlobalID': '{00000000-0000-0000-0000-000000000000}',
  'Uniq_Id': '',
  'State_Code': '9',
  'District_C': '144',
  'Tehsil_Cod': '742',
  'Block_Code': '1126',
  'Village_Co': '120196',
  'LGD_Code': '120196',
  'Built_Up_A': 1,
  'Village_Na': 'Kalonda',
  'Area_Desc': '',
  'Name': '',
  'Owner_Name': '',
  'Property_I': '',
  'Property_C': '',
  'Area_Sqm': 9.83786452932,
  'Roof_type': 1,
  'No_Floors': 1,
  'Remarks': '',
  'Adl_Info': '',
  'SHAPE_Leng': 17.252085894,
  'SHAPE_Area': 12.7944454348,
  'GP_Code': '63317',
  'GP_Name': 'KALONDA'},
 {'points': [{'X': 8646539.8134, 'Y': 3317220.3532999977},
   {'X': 8646538.384300001, 'Y': 3317219.28

In [13]:
for i in range(len(d)):
    print(len(d[i]["points"]))

6
6
10
11
5
5
12
6
8
8
5
7
15
7
5
6
8
5
6
6
11
8
10
8
10
5
7
5
5
7
8
5
7
9
5
9
8
8
5
5
7
5
13
13
7
5
6
13
6
5
10
5
8
7
5
7
12
5
6
5
7
6
7
7
7
10
7
9
5
5
5
5
5
6
11
5
8
10
10
7
5
5
5
7
5
5
6
7
5
5
5
5
9
12
19
5
5
6
9
5
11
9
5
7
5
9
10
8
6
8
6
7
9
9
14
7
5
7
7
5
5
5
5
5
8
7
9
10
10
7
7
5
5
6
7
5
6
5
5
9
9
9
8
5
5
5
5
7
6
7
8
5
5
6
9
5
5
11
9
5
7
6
6
6
10
10
5
7
8
5
5
6
6
14
5
7
6
6
7
7
5
5
8
8
5
6
5
6
7
6
5
14
5
6
8
5
7
5
7
11
7
11
11
5
12
9
6
7
6
7
8
6
5
5
5
7
7
11
10
5
5
5
6
5
10
6
5
5
7
7
5
6
6
5
6
6
6
6
7
10
7
9
5
5
6
6
7
13
5
5
11
5
5
5
5
6
10
10
6
7
5
7
5
12
10
5
5
7
5
5
5
7
5
5
10
13
6
5
6
8
8
6
9
5
7
7
8
5
5
6
5
5
5
6
5
5
5
7
6
6
5
7
8
9
5
7
5
5
6
11
5
5
7
8
5
6
6
7
5
5
9
10
5
8
5
6
8
10
8
5
7
6
6
5
5
6
8
5
5
5
5
7
9
5
8
5
15
5
5
5
5
5
5
6
6
7
5
6
11
6
6
14
5
11
8
7
5
5
8
6
8
6
5
5
5
5
5
7
5
5
6
5
19
11
9
5
8
6
7
5
5
7
7
7
6
5
8
5
19
5
6
8
5
8
8
5
7
6
7
6
7
5
9
10
6
5
5
8
6
8
5
5
6
9
9
7
7
6
5
8
7
8
9
10
9
10
5
13
13
5
5
9
9
6
5
5
7
5
6
8
5
5
7
5
5
8
5
9
8
6
5
6
7
5
7
6
5
5
6
6
8